# TP1: Procesamiento de Datos - Resuelto

Hay más de una manera de resolver cualquiera de estos ejercicios. Aquí algunas de las más naturales, sin rodeos, y en estilo "pythónico" como referencia futura para ustedes:

In [41]:
import pandas as pd

In [42]:
df = pd.read_csv("sube-2023.csv")

In [43]:
df.head()

,DIA_TRANSPORTE,NOMBRE_EMPRESA,LINEA,AMBA,TIPO_TRANSPORTE,JURISDICCION,PROVINCIA,MUNICIPIO,CANTIDAD,DATO_PRELIMINAR
0,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,1,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,61,NO
1,2023-01-01,MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...,2B,SI,COLECTIVO,MUNICIPAL,BUENOS AIRES,MERCEDES,11,NO
2,2023-01-01,EMPRESA BATAN S.A.,BS_AS_LINEA 715M,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1707,NO
3,2023-01-01,COMPAÑIA DE TRANSPORTE VECINAL S.A.,BS_AS_LINEA_326,SI,COLECTIVO,PROVINCIAL,BUENOS AIRES,SN,438,NO
4,2023-01-01,EMPRESA DE TRANSPORTE PERALTA RAMOS SACI,BS_AS_LINEA_512,NO,COLECTIVO,MUNICIPAL,BUENOS AIRES,GENERAL PUEYRREDON,1189,NO


## Punto 1

### (a) (i) Visualizar el tipo de datos de cada columna.
Sugerencia: investigar la función `to_datetime` de pandas. Para completar el argumento `format`, revisar la documentación de `datetime`.

En (i) el _atributo_ `dtypes` de un DataFrame cuenta con la información necesaria. El _método_ `info`, sin argumentos, es un poco más informativo.

En (ii), como la fecha _ya está en formato ISO 8601_, el estándar internacional más expandido, no hace falta especificar el argumento `format`. La referencia más clara sobre cómo interpretar texto como fechas está en la docu: [strftime() and strptime() Behavior](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)

In [141]:
# Visualizar el tipo de datos de cada columna.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472291 entries, 0 to 472290
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   DIA_TRANSPORTE   472291 non-null  datetime64[ns]
 1   NOMBRE_EMPRESA   472291 non-null  object        
 2   LINEA            472291 non-null  object        
 3   AMBA             472291 non-null  bool          
 4   TIPO_TRANSPORTE  472291 non-null  object        
 5   JURISDICCION     469742 non-null  object        
 6   PROVINCIA        469720 non-null  object        
 7   MUNICIPIO        469720 non-null  object        
 8   CANTIDAD         472291 non-null  int64         
 9   DATO_PRELIMINAR  472291 non-null  bool          
 10  FECHA_DIA        472291 non-null  object        
 11  FECHA_ORDINAL    472291 non-null  int64         
 12  FECHA_MES        472291 non-null  object        
dtypes: bool(2), datetime64[ns](1), int64(2), object(8)
memory usage: 40.5+ MB


In [142]:
df.dtypes

DIA_TRANSPORTE     datetime64[ns]
NOMBRE_EMPRESA             object
LINEA                      object
AMBA                         bool
TIPO_TRANSPORTE            object
JURISDICCION               object
PROVINCIA                  object
MUNICIPIO                  object
CANTIDAD                    int64
DATO_PRELIMINAR              bool
FECHA_DIA                  object
FECHA_ORDINAL               int64
FECHA_MES                  object
dtype: object

#### Un comentario sobre `dtypes`
Alguien escribió
```python
# Lo hacemos de esta manera, ya que si usamos el .dtype , nos dice que son todos objetos
for columna in sube2023.columns:
    print("la columna {0} tiene tipo {1}".format(columna, type(sube2023[columna][0])))
```

Mas allá de que no es cierto(`CANTIDAD` es de `dtype int` (entero). hay una diferencia entre el tipo de
1. una columna en el DataFrame (que es `pd.Series`),
1. el valor de una celda cualquiera en una columna, y
1. el tipo de datos del array de numpy subyacente a la serie.

El `dtype` de un `np.array`, será el tipo de datos más genérico que soporte a todos los elementos del array.
Vean:


In [143]:
s = pd.Series([True, 1, "uno", 1.2])
type(s), s.dtype, {str(v): type(v) for v in s.values}

(pandas.core.series.Series,
 dtype('O'),
 {'True': bool, '1': int, 'uno': str, '1.2': float})

Cuando no hay un `dtype` específico que soporte a todos los elementos, se defaultea al tipo `object` ("O"), pero cada elemento conserva su tipo original. Cuando sí existe tal tipo, pues se pueden _coercionar_ los distintos elementos a algo más específico sí que todos los elementos lo toman:

In [144]:
s = pd.Series([1.2, 12])
type(s), s.dtype, {str(v): type(v) for v in s.values}

(pandas.core.series.Series,
 dtype('float64'),
 {'1.2': numpy.float64, '12.0': numpy.float64})

 `object`, es, de hecho, es el tipo por default para `string`s (y todo otro objeto arbitrario): de la [docu](https://pandas.pydata.org/docs/user_guide/text.html),
> For backwards-compatibility, `object` dtype remains the default type we infer a list of strings to



### (ii) Transformar la columna DIA_TRANSPORTE para que sea reconocida como una fecha.
Cuando un array de numpy tiene un formato de fecha así de prolijo, se lo puede _castear_ a `datetime` cambiándole el tipo directamente, con `astype("datetime64[ns]")`.

In [46]:
assert df.DIA_TRANSPORTE.astype("datetime64[ns]").equals(
    pd.to_datetime(df.DIA_TRANSPORTE)
)
df["DIA_TRANSPORTE"] = pd.to_datetime(df.DIA_TRANSPORTE)

### (b) Agregar tres columnas al DataFrame:

1. `FECHA_DIA` : debe indicar el nombre del día de la semana correspondiente a `DIA_TRANSPORTE`
1. `FECHA_ORDINAL` : debe indicar el ordinal correspondiente a `DIA_TRANSPORTE` (por ejemplo, a 2023-01-01 le corresponde 1, a 2023-01-02 le corresponde 2 y así sucesivamente). Debe ser un entero (int).
1. `FECHA_MES` : debe indicar el [nombre|número] mes correspondiente a `DIA_TRANSPORTE`.

Sugerencia: investigar el método `apply` de DataFrame

#### "Accesor" `pd.Series.dt`
La sugerencia de `apply`, aunque técnicamente correcta y alineada con lo visto en clase, no es la forma más directa de resolver estas cuestiones. De la misma manera que `pd.Series.str` provee acceso a todos los métodos comunes del tipo `str` (cadena de texto) pero vectorizados sobre la serie completa, `pd.Series.dt` ([docs](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.html)) provee el acceso equivalente para los objetos de tipo `datetime`. En general, si `s` es una serie con `dtype` `datetime`, `s.dt.metodo_dt(...)` es equivalente a `s.apply(lambda x: x.metodo_dt(...))`, y ligeramente más rápido.

Para todas estas consignas, hay al menos dos formas de hacerlo, además de la amplia creatividad del alumnado. O bien con un método de `datetime` directamente, o bien con la navaja suiza de formateo de fechas, `strftime` y una string de formato adecuada. Ambas son igual de válidas.

#### FECHA_DIA

In [47]:
df["FECHA_DIA"] = df.DIA_TRANSPORTE.dt.strftime("%A")
df.FECHA_DIA.value_counts()

FECHA_DIA
miércoles    70432
jueves       69989
martes       69812
viernes      69405
lunes        68572
sábado       64141
domingo      59940
Name: count, dtype: int64


#### Localización
Verán que por defecto, los _nombres_ de días y meses están en inglés (o el idioma de su sistema operativo, si lo han configurado). Para que Python los _localice_ (i.e., los represente en el idioma del usuario), hay que setear correctamente el "local" de nuestro código. Una vez que se setea el "[locale](https://docs.python.org/3/library/locale.html)", `strftime` siempre lo respetará. A los métodos específicos de `pd.Series.dt`, hay que especificárselo en cada llamada, pues el "default" es `"en_US.utf8"`.


In [48]:
import locale

# https://docs.python.org/3/library/locale.html#locale.setlocale
locale.setlocale(locale.LC_ALL, "es_ES")

'es_ES'

In [49]:
df["FECHA_DIA"] = df.DIA_TRANSPORTE.dt.strftime("%A")
df.FECHA_DIA.value_counts()

FECHA_DIA
miércoles    70432
jueves       69989
martes       69812
viernes      69405
lunes        68572
sábado       64141
domingo      59940
Name: count, dtype: int64

In [50]:
df.DIA_TRANSPORTE.dt.day_name(locale="es_ES").value_counts()

DIA_TRANSPORTE
Miércoles    70432
Jueves       69989
Martes       69812
Viernes      69405
Lunes        68572
Sábado       64141
Domingo      59940
Name: count, dtype: int64

#### FECHA_ORDINAL
Para `FECHA_ORDINAL`, muchos dieron el día relativo al mes, pero el "por ejemplo" habla del primer día del año (`2023-01-01`), y no del primer día de un mes. `"%j%` para `strftime` (escribir fechas) y `strptime` (leer/interpretar fechas) y `dt.dayofyear` cumplen la función.

In [51]:
df["FECHA_ORDINAL"] = df.DIA_TRANSPORTE.dt.strftime("%j").astype(int)
assert (df.FECHA_ORDINAL == df.DIA_TRANSPORTE.dt.dayofyear).all()


#### Otras resoluciones

##### `datetime.toordinal()`

Hubo quien hizo:


In [52]:
df["FECHA_ORDINAL"] = df["DIA_TRANSPORTE"].apply(lambda x: x.toordinal()) - 738520
# O aún
df["FECHA_ORDINAL"] = df["DIA_TRANSPORTE"].apply(lambda x: x.toordinal()) - (
    df.DIA_TRANSPORTE.min().toordinal() + 1
)

Aquí el método es adecuado, pero peligroso por el "hard-coding" del "día cero", a un entero fijo, o hacerlo dependiente del `DataFrame` evaluado. ¿Y si no había datos para el 1 de Enero? Una opción, sería:

In [53]:
import datetime as dt
dia_cero = dt.date(2022, 12, 31).toordinal()
df["FECHA_ORDINAL"] = df.DIA_TRANSPORTE.apply(dt.date.toordinal) - dia_cero

In [54]:
assert (738520 == df.DIA_TRANSPORTE.min().toordinal() - 1) and (
    738520 == dia_cero  # 31/12/22
)

Esta versión es especialmente útil cuando hay que numerar días cptes. a períodos interanuales, o que no arrancan el 1/1 de un año.

##### Funciones de conteo
Algunos "reinventaron la rueda" con su propia función de conteo.

In [55]:
def fecha_ordinal(fecha):
    dias_meses = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    mes = fecha.month
    dia = fecha.day
    ordinal = 0
    for i in range(mes - 1):
        ordinal += dias_meses[i]
    ordinal += dia
    return ordinal


Peligrosa práctica: para cosas tan comunes, de seguro algún ingeniero bien pago ya lo hizo mejor que nosotros, y lo agregó a la librería básica.
¿Qué pasa si el año es bisiesto? ¿Si el año es previo al calendario gregoriano? Parece mentira, pero esas cosas pasan, y ya están contempladas en las funciones estándares.

##### `date.timetuple().tm_yday`
Esta no la me la sabía. Resulta que `date` tiene un método [`timetuple`](https://docs.python.org/3/library/datetime.html#datetime.date.timetuple) que expresa la fecha como un [`time.struct_time`](https://docs.python.org/3/library/time.html#time.struct_time), una "tupla temporal", que es una representación numérica bastante útil. Un poco arcana, pero válida. Como `tm_yday` es atributo de la tupla temporal que devuelve `date.timetuple()` y no del objeto Datetime, aquí sí se hace necesario definir un método que pasarle a `apply`. Recuerden, en general, que es preferible definir el método brevemente y no en un `lambda` "al vuelo":

In [56]:
def get_fecha_ordinal(date):
  return date.timetuple().tm_yday

df["FECHA_ORDINAL"] = df.DIA_TRANSPORTE.apply(get_fecha_ordinal)

#### FECHA_MES
Por como está escrita la consigna, podría haber sido tanto el nombre como el número de mes lo solicitado. Por analogía con FECHA_DIA, uno esperaría que se pide el nombre, pero ambas valen.

In [57]:
# Como número del mes
assert (
    df.DIA_TRANSPORTE.dt.month == df.DIA_TRANSPORTE.dt.strftime("%m").astype(int)
).all()
# Como nombre del mes
assert (
    df.DIA_TRANSPORTE.dt.month_name(locale="es_ES")
    == df.DIA_TRANSPORTE.dt.strftime("%B").str.capitalize()
).all()

df["FECHA_MES"]= df.DIA_TRANSPORTE.dt.month_name(locale="es_ES")

## Punto 2
### Crear el DataFrame datos_amba, el cual sólo debe tener datos de AMBA y debe excluir datos preliminares. Además, al ejecutar datos_amba.head() debe observarse el siguiente orden y formato de columnas:

![](img/tp1-pt1-ej2.png)

Imagino que de apurados, pero muchísima gente se olvidó de descartar los "datos preliminares" (`df.DATO_PRELIMINAR == "SI"`). Fuera de eso, el ejercicio era casi trivial.

Como localizamos los nombres, se van a ver algo distintos a los del data farme de ejemplo, pero mejor 😉. No se descontaron puntos por estas discrepancias de estilo.

In [58]:
df[["AMBA", "DATO_PRELIMINAR"]].value_counts(dropna=False)

AMBA  DATO_PRELIMINAR
NO    NO                 322250
SI    NO                 149973
      SI                     41
NO    SI                     27
Name: count, dtype: int64

In [59]:
datos_amba = df[(df.AMBA == "SI") & (df.DATO_PRELIMINAR == "NO")]
datos_amba = datos_amba.rename(
    columns={"DIA_TRANSPORTE": "FECHA", "CANTIDAD": "PASAJEROS"}
).rename(columns=str.lower)
cols = [
    "fecha",
    "fecha_dia",
    "fecha_mes",
    "fecha_ordinal",
    "jurisdiccion",
    "linea",
    "pasajeros",
    "tipo_transporte",
]
datos_amba = datos_amba[cols]
datos_amba.head()

,fecha,fecha_dia,fecha_mes,fecha_ordinal,jurisdiccion,linea,pasajeros,tipo_transporte
0,2023-01-01,domingo,Enero,1,MUNICIPAL,1,61,COLECTIVO
1,2023-01-01,domingo,Enero,1,MUNICIPAL,2B,11,COLECTIVO
3,2023-01-01,domingo,Enero,1,PROVINCIAL,BS_AS_LINEA_326,438,COLECTIVO
5,2023-01-01,domingo,Enero,1,MUNICIPAL,BS_AS_LINEA_514,3067,COLECTIVO
6,2023-01-01,domingo,Enero,1,MUNICIPAL,BS_AS_LINEA_522,332,COLECTIVO


### Operaciones lógicas vectorizadas: [np.logical_and](https://numpy.org/doc/stable/reference/generated/numpy.logical_and.html) et ales
Las operaciones lógicas binarias clásicas, `AND, OR, XOR (== OR excluyente)` y la "unaria" `NOT` tienen sus equivalentes en `numpy`, y sus versiones taquigráficas:
- AND: `np.logical_and` o `&`
- OR: `np.logical_or` o `|`
- NOT: `np.logical_not` o `~`
- XOR: `np.logical_xor`

Luego, expresiones del tipo

In [60]:
df2 = df[df.AMBA == "SI"]
df2 = df2[df2.DATO_PRELIMINAR == "NO"]

Se escriben más concisamente como

In [61]:
df2 = df[(df.AMBA == "SI") & (df.DATO_PRELIMINAR == "NO")]

Además, permiten expresar condiciones complejas como "las filas cptes. al AMBA o cuyo tipo de transporte es COLECTIVO pero cuentan con menos de 500 pasajeros". Nótese el uso adecuado de paréntesis, como en IPC del CBC.


In [62]:
df2 = df[
    ((df.AMBA == "SI") | (df.TIPO_TRANSPORTE == "COLECTIVO")) & (df.CANTIDAD < 500)
]

### Transformar SI/NO en booleano
Para algo tan puntual, podemos dejar las strings de texto como tales, pero de trabajar mucho con lso datos, mejor pasarlos a su `dtype` natural, `boolean`:

In [63]:
bool_cols = ["AMBA", "DATO_PRELIMINAR"]
df3 = df
df3[bool_cols] = df3[bool_cols].eq("SI")
df3[bool_cols].value_counts()

AMBA   DATO_PRELIMINAR
False  False              322250
True   False              149973
       True                   41
False  True                   27
Name: count, dtype: int64

Luego, la condición de base del ejercicio queda más elegantemente expresada como `df[df.AMBA & ~df.DATO_PRELIMINAR]`.

## Punto 3
### Utilizando `datos_amba`, identificar: (a) la proporción de la cantidad total anual de pasajeros que le corresponde a cada medio de transporte

In [64]:
pasajeros_por_medio = datos_amba.groupby("tipo_transporte").pasajeros.sum()
pasajeros_por_medio / pasajeros_por_medio.sum()

tipo_transporte
COLECTIVO    0.837645
SUBTE        0.068097
TREN         0.094258
Name: pasajeros, dtype: float64

Con [`transform`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.transform.html), prima de `aggregate`, podemos encadenar ambas operaciones.

In [66]:
datos_amba.groupby("tipo_transporte").pasajeros.sum().transform(lambda x: x / x.sum())

tipo_transporte
COLECTIVO    0.837645
SUBTE        0.068097
TREN         0.094258
Name: pasajeros, dtype: float64

### (b) la tupla (mes, línea de subte) donde viajó la mayor cantidad de pasajeros

In [67]:
datos_amba[datos_amba.tipo_transporte == "SUBTE"].groupby(
    ["fecha_mes", "linea"]
).pasajeros.sum().idxmax()

('Agosto', 'LINEA_B')

Recuerden que si agrupan por varios valores a la vez, el `MultiIndex` resultante es una tupla, así que se puede hacer:

In [75]:
datos_amba.query("tipo_transporte == 'SUBTE'").groupby(
    ["fecha_mes", "linea"]
).pasajeros.sum().sort_values(ascending=False).index[0]

('Agosto', 'LINEA_B')

Un error muy común fue devolver los valores de `(mes, línea)` correspondientes al registro con más pasajeros, una lectura detenida del ejercicio no avala dicha interpretación.

### (c) el día hábil con menor desvío estándar en cantidad de pasajeros
Esta pregunta resultó algo ambigua, pues no era del todo claro cuál era la unidad de análisis. Una interpretación estricta, agruparía los datos al nivel de "día" (`FECHA`) y sumaría los pasajeros de todos los tipos y líneas de transporte para cada día. Sobre esos datos, se quedaría con "los días hábiles", los agruparía por nombre de día, de cada grupo tomaría el desvió, y se quedaría con el nombre del día hábil de menor variación.

Esta operación se puede realizar encadenando operaciones, pero implica una cadena de "agrupado - agregación - reagrupado - reagregación" que puede confundir. Mejor, en partes:

In [99]:
datos_amba_semana = datos_amba[~datos_amba.fecha_dia.isin(["sábado", "domingo"])]
pasajeros_por_dia = datos_amba_semana.groupby("fecha").pasajeros.sum().reset_index()
pasajeros_por_dia.sample(5)

,fecha,pasajeros
220,2023-11-06,12899182
75,2023-04-17,12332431
150,2023-07-31,11620117
50,2023-03-13,11150866
192,2023-09-27,12988479


In [100]:
desvio_por_dia = pasajeros_por_dia.groupby(
    pasajeros_por_dia.fecha.dt.strftime("%A")
).pasajeros.std()
desvio_por_dia.sort_values(), desvio_por_dia.idxmin()

(fecha
 miércoles    1.274548e+06
 jueves       1.861006e+06
 martes       1.951223e+06
 viernes      2.625816e+06
 lunes        2.952116e+06
 Name: pasajeros, dtype: float64,
 'miércoles')

Como agrupar por `fecha` y `(fecha, fecha_dia)` dan la misma cantidad de grupos únicos (cada `fecha` tiene exactamente un valor de `fecha_dia` asociado), y **se puede agrupar un dataframe con los nombres de los niveles del índice**, las operaciones previas se pueden encadenar así:

In [108]:
datos_amba_semana.groupby(["fecha_dia", "fecha"]).pasajeros.sum().groupby(
    "fecha_dia"
).std().round(2)

fecha_dia
jueves       1861006.25
lunes        2952115.90
martes       1951222.63
miércoles    1274548.09
viernes      2625815.94
Name: pasajeros, dtype: float64

La enorme mayoría - yo incluido en la primera resolución apurado del TP - simplemente tomaron como unidad de análisis cada fila, agrupándolas por `fecha_dia`. El problema de este enfoque, es que implica mezclar peras con manzanas: una línea de tren no lleva la cantidad de gente de una de colectivos, por ejemplo. Aquí, dos resoluciones que también dimos por válidas: 
1. tomando cada fila (`fecha, tipo_transporte, linea`) como unidad de análisis, y calculando el desvío estándar global, 
2. y una con la misma unidad, pero tomando el desvío estándar por tipo_transporte:

In [111]:
desvios = datos_amba_semana.groupby("fecha_dia").pasajeros.agg("std")
desvios.round(2), desvios.idxmin()

(fecha_dia
 jueves       35968.32
 lunes        33577.46
 martes       35833.74
 miércoles    35980.85
 viernes      35059.69
 Name: pasajeros, dtype: float64,
 'lunes')

In [115]:
desvios_por_tipo = datos_amba_semana.groupby(
    ["tipo_transporte", "fecha_dia", "fecha"]
).pasajeros.sum().groupby(["tipo_transporte", "fecha_dia"]).agg(
    "std"
).reset_index().pivot(
    index="tipo_transporte", columns="fecha_dia"
)
desvios_por_tipo.round(2)

pasajeros                                                
fecha_dia            jueves       lunes      martes   miércoles     viernes
tipo_transporte                                                            
COLECTIVO        1572568.07  2446883.68  1624582.67  1110180.13  2261089.79
SUBTE             149644.40   217443.15   155885.99   100198.91   171534.97
TREN              178726.31   311527.10   197319.73   109293.41   251264.65

In [118]:
desvios_por_tipo.idxmin(axis=1)

tipo_transporte
COLECTIVO    (pasajeros, miércoles)
SUBTE        (pasajeros, miércoles)
TREN         (pasajeros, miércoles)
dtype: object

Noten que tanto en (1) como (2), los desvíos son mucho más parecidos entre días hábiles. Intuitivamente, pareciera más coherente la resolución original: de entre los días hábiles, el miércoles es el feriado menos común, por ejemplo, así que tiene una cantidad más constante de pasajeros.

## Comentarios Generales
A continuación, unas cuantas correcciones y sugerencias sin nombre, que aplican a una buena parte de los trabajos presentados. Revísenlas, aunque no hayan cometido ninguna, de seguro hay algo para aprender. Los pedazos de código están adaptados de lo visto en los trabajos.

#### 🟡 Es redundante descartar columnas si más adelante específicamente seleccionan el complemento
Aquí debajo, `drop` es en vano; toda columna que no figure en `nuevo_orden`, que se le pasan a `reindex` en la línea siguiente, será descartada:
```python
datos_amba.drop(columns=['dato_preliminar','nombre_empresa','municipio','amba','provincia'], inplace=True)
nuevo_orden = ['fecha','fecha_dia','fecha_mes','fecha_ordinal','jurisdiccion','linea','pasajeros','tipo_transporte']
datos_amba = datos_amba.reindex(columns=nuevo_orden)
```

#### 🟡 No descarten información 'para complacer al docente'
Entiendo que si el ejemplo dice `"Sun"`, y uno encontró un código que devuelve `"Sunday"`, la deferencia a la autoridad quiera hacer encajar el código con el ejemplo a toda costa, pero operaciones así son "de pura pinta"
```
datos_amba['fecha_dia'] = datos_amba['fecha_dia'].str.slice(0, 3)
```
En cualquier caso, si llegaron a representar `"Sunday"` con `df.dia_transporte.dt.strftime("%A")` (nombre completo del día), también pueden leer la línea inmediata superior de la documentación y usar `"%a"` (3 primeras letras del nombre del día), si quieren hacerlo al dedillo.


#### 🔴 Usen groupby para replicar operaciones con distintos grupos

```python
desvios = []
dias_habiles = datos_amba_habiles.fecha_dia.unique()
for dia_habil in dias_habiles:
    desvio_estandar = datos_amba_habiles[datos_amba_habiles['fecha_dia']==dia_habil]['pasajeros'].std().round(1)
    desvio_dia = (dia_habil, desvio_estandar)
    desvios.append(desvio_dia)
```

Literalmente, `pd.DataFrame.groupby(key)`, parte los datos de `DataFrame` según los valores únicos de la columna `key` y devuelve un iterador sobre los pares `clave, df`. Vean:

In [125]:
for dia, data in datos_amba.groupby("fecha_dia"):
    print(dia, type(data), data.shape)

domingo <class 'pandas.core.frame.DataFrame'> (21350, 8)
jueves <class 'pandas.core.frame.DataFrame'> (21476, 8)
lunes <class 'pandas.core.frame.DataFrame'> (21415, 8)
martes <class 'pandas.core.frame.DataFrame'> (21445, 8)
miércoles <class 'pandas.core.frame.DataFrame'> (21468, 8)
sábado <class 'pandas.core.frame.DataFrame'> (21370, 8)
viernes <class 'pandas.core.frame.DataFrame'> (21449, 8)


Los data frames en que queda partido `datos_amba` tienen la misma cantidad de columnas, y tantas filas como ocurrencias únicas tenga la clave de partición. Por sobre todo, cada vez que intentan implementar a mano una función básica de la librería, corren riesgos de implementarla mal. Lean la documentación, y resuelvan, por ejemplo, con

```python
datos_amba_habiles.groupby("fecha_dia").pasajeros.std()
```

¿No es mucho más legible?

#### 🔴 Eviten "inspeccionar a ojo los datos"
Si se pide un mínimo, o el valor que minimiza una variable, usen `min` e `idxmin` respectiavmente. Aunque con pocas categorías (como 5 días hábiles) es factible hacerlo a mano, con más categorías no es plausible, y sí es propenso a errores.

#### 🟡 Los títulos en Markdown lelvan espacio entre los marcadores de nivel (#) y el texto
El notebook que entregan es un documento oficial, preséntenlo como presentarían un TP escrito. De mínima, asegúrense que los títulos se vean como tales. La [guía de GitHub](https://docs.github.com/es/get-started/writing-on-github/getting-started-with-writing-and-formatting-on-github/basic-writing-and-formatting-syntax), que está en espannol también, es una referencia decente para empezar.


#### 🔴 Nunca entreguen código que no corre
Hubo quienes llamaban `df = pd.read_csv("datos.csv")` sin importar `pandas` primero, o mencionaban un archivo extra que luego no adjuntaban. Evitar a toda costa: un notebook tiene que poder correr "de pe a pa", con a lo sumo un cambio de ruta para que el docente pueda apuntar la lectura de archivos a sus directorios locales.

#### 🟡 Salvo que usen `inplace=True`, `drop` _no elimina columnas más que en el retorno de la función.

```python
# Esto no hace absolutamente nada sobre datos_amba
datos_amba.drop(['AMBA','DATO_PRELIMINAR','NOMBRE_EMPRESA'], axis=1)
```

#### 🟡 Para copiar un df, hay que usar `copy()` explícitamente:
Si no, sólo obtiene una nueva referencia al mismo objeto
```python
# Así no:
datos_amba_copia = datos_amba
# Así sí:
datos_amba_copia = datos_amba.copy()
```

#### 🟡 Para mostrar datos, `pd.Series.to_dict` solo empeora la presentacion sin aportar nada por sobre la representación tradicional:

In [145]:
s = df.groupby("TIPO_TRANSPORTE").CANTIDAD.sum()
display(s), display(s.to_dict())

TIPO_TRANSPORTE
COLECTIVO    3837936036
LANCHAS          205543
SUBTE         239621028
TREN          331984197
Name: CANTIDAD, dtype: int64

{'COLECTIVO': 3837936036,
 'LANCHAS': 205543,
 'SUBTE': 239621028,
 'TREN': 331984197}

(None, None)

#### 🟡 `pd.DataFrame.rename` acepta un diccionario con tantas entradas como uno desee para renombrar un eje

```python

# Redundante
datos_amba = datos_amba.rename(columns={"DIA_TRANSPORTE": "fecha"})
datos_amba = datos_amba.rename(columns={"CANTIDAD": "pasajeros"})

# Mejor
datos_amba = datos_amba.rename(
    columns={"DIA_TRANSPORTE": "fecha", "CANTIDAD": "pasajeros"}
)
```

####  🔴 Sobreescribir nombres de funciones estándares es una mala idea en general
Los mensajes de error son crípticos más adelante, y no es fácil de enmendar:

In [146]:
max([2, 5, 4])

5

In [147]:
max = datos_amba.pasajeros.max()
print(max)

542616


In [148]:
max([2, 5, 4])

TypeError: 'numpy.int64' object is not callable

## Sobreingeniería
Más que errores hechos y derechos, lo que abunda en los TPs es la sobreingeniería ("overengineering"): máquinas de [Rube Goldberg](https://www.youtube.com/watch?v=qybUFnY7Y8w) enormemente complicadas cuando existe un enfoque más directo. Aquí el problema es doble:
- se agranda la superficie de riesgo para cometer errores, y además
- se gasta el "presupuesto cognitivo" del lector en trivialidades en lugar de reservarlo para casos realmente necesarios.

Cuando programen, no escriban "para la computadora", escriban para el humano que lo va a leer (que típicamente, serán ustedes mismos más tarde), y valoren su tiempo y esfuerzo como el propio. A continuación, algunos ejemplos al azar,


#### No escriben funciones de único uso, ni asignen a variables datos que no van a reutilizar

```python
# Esta función no se uso nunca más
def descartaNegativos(df):
    df.loc[df['CANTIDAD'] >= 0]  # Por qué lo llaman en el vacío?
    df = df.loc[df['CANTIDAD'] >= 0]
    return df

sube = descartaNegativos(sube)
# Mejor,
sube = sube[sube.CANTIDAD >= 0]

# Lo mismo aplica a las funciones anónimas o "lambdas"
datos_amba.rename(columns=(lambda x: x.lower()))
# Mejor,
datos_amba.rename(columns=str.lower)
```

Tampoco vale la pena asignar algo que se va a mostrar inmediatamente después:
```python
# Así no:
dtypes = df.dtypes
dtypes
# Así sí:
df.dtypes

# Así no:
amba = sube['AMBA'] == 'SI'
datos_amba = sube[amba]
# Así sí:
datos_amba = sube[sube.AMBA == 'SI']

# Así no:
FECHA_DIA= datos['DIA_TRANSPORTE'].apply(lambda x: x.strftime('%a'))
datos['FECHA_DIA']=FECHA_DIA
# Así sí:
datos['FECHA_DIA']=datos['DIA_TRANSPORTE'].apply(lambda x: x.strftime('%a'))
```




#### Ojo con dar vueltas en círculos para llegar adonde empezaron
```python
# Así no:
FECHA_DIA = df["DIA_TRANSPORTE"]
df["FECHA_DIA"] = FECHA_DIA
df["FECHA_DIA"] = pd.to_datetime(df["FECHA_DIA"])
df["FECHA_DIA"] = df["FECHA_DIA"].dt.strftime("%A")
# Así sí:
df["FECHA_DIA"] = df["DIA_TRANSPORTE"].dt.strftime("%A")
```

#### `pd.Series.map` es su amigo para renombrar celdas
La operación de mapear laos valores de una serie a un diccionario (como un `rename` pero para los _valores_ de un DataFrame, no sus indices/columnas), se llama `map`. Si no, la función para tomar el valor cpte. a una clave de un `dict` diccionario (o `None` por defecto), es `dict.get`:

```python
NombreDiasIngles = df["DIA_TRANSPORTE"].apply(lambda x: x.strftime("%A"))
diccionario = {
    "Monday": "Lunes",
    "Tuesday": "Martes",
    "Wednesday": "Miércoles",
    "Thursday": "Jueves",
    "Friday": "Viernes",
    "Saturday": "Sábado",
    "Sunday": "Domingo",
}

# Innecesaria función auxiliar:
def diccionario_dias_ingles_a_espanol(n):
    return diccionario[n]
df["FECHA_DIA"] = NombreDiasIngles.apply(diccionario_dias_ingles_a_espanol)
# Sin la función:
df["FECHA_DIA"] = NombreDiasIngles.apply(diccionario.get)
# Aún mejor
df["FECHA_DIA"] = NombreDiasIngles.map(diccionario)
```

### DRY: Don't Repeat Yourself ("No Te Repitas")
[Wikipedia](https://es.wikipedia.org/wiki/No_te_repitas) lo explica bien, pero tl; dr: si tienen que hacer la misma operación dos veces, la abstraen a una función, así en el futuro, de tener que modificarla, la modifican en un solo lugar. Este principio, obviamente, cede en importancia al más fundamental: si ya está implementada, no la reimplementen.

Una extensión: si el código es obvio de leer, no hace falta comentarlo, pues sería repetir lo uqe ya dice el código. Limítense a comentar lo que no resulta obvio de leer el código, y ejecuten la moderación aún en ello.

Vean esto:
```python
# Filtramos en el dataset en cada dia hábil y calculamos sus desvios estandar
datos_amba_lunes_std = datos_amba[datos_amba["fecha_dia"] == "Lunes"][
    "pasajeros"
].std()  # Lunes
datos_amba_Martes_std = datos_amba[datos_amba["fecha_dia"] == "Martes"][
    "pasajeros"
].std()  # Martes
datos_amba_Miércoles_std = datos_amba[
    datos_amba["fecha_dia"] == "Miércoles"
][
    "pasajeros"
].std()  # Miércoles
datos_amba_Jueves_std = datos_amba[datos_amba["fecha_dia"] == "Jueves"][
    "pasajeros"
].std()  # Jueves
datos_amba_Viernes_std = datos_amba[datos_amba["fecha_dia"] == "Viernes"][
    "pasajeros"
].std()  # Viernes

# Armamos una tupla con los desvíos estándar calculados y elijimos el minimo valor
datos_amba_tupla_d_v_dias_habiles = min(
    datos_amba_lunes_std,
    datos_amba_Martes_std,
    datos_amba_Miércoles_std,
    datos_amba_Jueves_std,
    datos_amba_Viernes_std,
)

# Comprobamos a qué día hábil pertenece ese valor
print("Desvio estandar minimo = ", datos_amba_tupla_d_v_dias_habiles)

# Armamos un dataframe para verlo mejor

datos_desvios_estandar = {
    "Día": ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes"],
    "Desvío Estándar": [
        datos_amba_lunes_std,
        datos_amba_Martes_std,
        datos_amba_Miércoles_std,
        datos_amba_Jueves_std,
        datos_amba_Viernes_std,
    ],
}
df_desvios_estandar = pd.DataFrame(datos_desvios_estandar)

print(df_desvios_estandar)
```

Y compárenlo con esta línea:
```python
datos_amba.query("fecha_dia not in ('sábado', 'domingo')").groupby(
    "fecha_dia"
).pasajeros.std()
```

¿Qué les resulta más fácil de seguir?

### Codificación
Si se encuentran con que la localización (AKA "internacionalización", o "traducción al idioma local") genera caracteres raros en letras con tildes o diéresis, como "miÃ©rcoles" en lugar de "miércoles", es porque están usando una codificación incorrecta en su sistema. 

In [150]:
s = "Martín Miguel de Güemes".encode("utf-8")
display(s)  # codificación en bytes, usando Unicode: los caracteres ascii son idénticos a su codificación, otros no.

b'Mart\xc3\xadn Miguel de G\xc3\xbcemes'

In [151]:
display(s.decode("utf-8"))  # decodificación en el mismo código


'Martín Miguel de Güemes'

In [152]:
display(s.decode("latin-1"))  # decodificación con un mapa de caracteres ligeramente diferente; noten qué caracteres fallan

'MartÃ\xadn Miguel de GÃ¼emes'

La codificación por defecto en Python 3 (y casi todo sistema moderno) es "utf-8" ([wikipedia](https://es.wikipedia.org/wiki/UTF-8)); por razones históricas Windows usa "latin-1" ([wikipedia](https://es.wikipedia.org/wiki/ISO/IEC_8859-1)).

## Bonus
### Black, un "linter" automático
Un "linter" ("sacapelusa", literalmente) es una herramienta que aplica ciertas reglas duras para formatear el código que ustedes escriban automágicamente. Todo este notebook, por ejemplo, está "linteado" con [Black](https://black.readthedocs.io/en/stable/), que es tan "opinionado" que no tiene ningún parámetro configurable por defecto. Algunos dirán: ¿por qué tendría que ceñirme tanto a especificaciones arbitrarias? A lo qu otros respondemos: ¿no es hermoso olvidarse de cómo tiene que lucir, y simplemente preocuparse de que corra?

Black aplica minuciosamente unas cuantas reglas que la comunidad de _pythonistas_ ha ido consensuando a lo largo de los años, y está bien implementado en cualquier editor de texto moderno. Les recomiendo ampliamente que lo prueben.

### La keyword `match`
En `python 3.10` se introdujo una _keyword_ nueva, `match`, que viene a ser algo así como un `case` con esteroides. La propuesta está bien explicada en el [PEP 636](https://peps.python.org/pep-0636/) (Python Enhancement Proposal). Honestamente, no la conocía hasta que la vi en uno de los TPs. Recomiendo chusmearla para quienes tengan ganas de probar algo de python "bien moderno".

En este caso, el mismo objetivo se podía lograr con un simple diccionario, o una serie de `if ... elif ... elif ... else`, pero dejo el código para que vean la idea:

```python
datos['FECHA_DIA'] = datos["DIA_TRANSPORTE"].dt.weekday
def diafecha(dato):
    match dato:
        case 6:
            return 'Domingo'
        case 0:
            return 'Lunes'
        case 1:
            return 'Martes'
        case 2:
            return 'Miercoles'
        case 3:
            return 'Jueves'
        case 4:
            return 'Viernes'
        case 5:
            return 'Sabado'


datos['FECHA_DIA'] = datos['FECHA_DIA'].map(diafecha)
```